In [23]:
import pandas as pd
import seaborn as sns
from datetime import datetime, date
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
os.chdir("/Users/wvsharber/FlatironLessons/mod3project_Tanzania_water_well/")
from src.useful_func import do_everything, create_voting_classifier, create_graph, clean_data, drop_useless_cols, fix_dates, clean_data, bin_me, onehotmess
os.chdir("/Users/wvsharber/FlatironLessons/mod3project_Tanzania_water_well/competition/")

In [15]:
string_1 = '../data/training_data_values.csv' #Your filepath to the data here
string_2 = '../data/training_data_labels.csv' #Your filepath to the data here
drop_columns = ['id', 'funder', 'num_private', 'longitude', 'latitude', 'wpt_name', 'subvillage', 'region_code', 'lga',
                'ward','recorded_by', 'scheme_name', 'extraction_type_group', 'payment', 'quality_group', 
                'quantity_group', 'source_type', 'source_class', 'waterpoint_type_group']


X_train, X_test, y_train, y_test, df_values, df_target = do_everything(string_1, string_2, drop_columns)

In [24]:
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [19]:
RF1 = RandomForestClassifier(n_estimators = 200)
LR1 = LogisticRegression(max_iter = 1000)
GBR1 = GradientBoostingClassifier(n_estimators = 300)
KNN1 = KNeighborsClassifier()

#Hard-coded estimates of accuracy from previously fit models
lr_weight, rf_weight, gbr_weight, knn_weight = .77, .80, .78, .78

eclf_soft = VotingClassifier(estimators = [('lr', LR1),
                                           ('rf', RF1),
                                           ('gbr', GBR1),
                                           ('knn', KNN1)],
                             weights = [lr_weight, rf_weight, gbr_weight, knn_weight],
                             voting = 'soft')
eclf_soft.fit(X_train, y_train)
score = eclf_soft.score(X_test, y_test)

In [21]:
test_df = pd.read_csv("../data/test_data_values.csv")

In [27]:
thresh = 200

df_dropped = drop_useless_cols(test_df, drop_columns)
fixed_date = fix_dates(df_dropped)
cleaner_df = clean_data(fixed_date, thresh)
df_binned = bin_me(cleaner_df)
df_objects = df_binned.select_dtypes(exclude=['int','float'])
df_nums = df_binned.select_dtypes(include=['int','float'])
df_onehot = pd.get_dummies(df_objects)
df_final = pd.concat([df_nums, df_onehot], axis = 1)

In [32]:
X_final = np.array(df_final)

In [35]:
X_train.shape

(56430, 292)

In [36]:
X_final.shape

(14850, 185)

In [34]:
X_train.shape == X_final.shape

False

In [33]:
test_data = scaler.transform(X_final)

ValueError: operands could not be broadcast together with shapes (14850,185) (292,) (14850,185) 